In [3]:
!python -m pip install -q transformers
!python -m pip install -q pillow
!python -m pip install -q loguru
!python -m pip install -q pydantic
!python -m pip install -q pandas
!python -m pip install -q opencv-python
!python -m pip install -q optimum[onnxruntime]
!python -m pip install -q supervision


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from PIL import Image
import torch
import cv2

from optimum.onnxruntime import ORTModelForCustomTasks
from transformers import CLIPTokenizerFast,OwlViTImageProcessor

In [5]:
def read_video(video_path, skip_frames=0):
    cap = cv2.VideoCapture(video_path)
    frame_id = 0

    if not cap.isOpened():
        raise Exception(f"Error: Could not open video {video_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        yield frame_id, frame

        if not skip_frames:
            frame_id += 1

        # Skip the specified number of frames and maintain frame_id
        for _ in range(skip_frames):
            ret = cap.grab()  # Just grab the frame without decoding
            if ret:
                frame_id += 1
            else:
                break

    cap.release()



In [6]:
vid_file = "./data/1080.mp4"
video = {frame_id: frame for  frame_id, frame in read_video(vid_file)}
len(video)

283

In [7]:
w,h = video[0].shape[1], video[0].shape[0]
target_size=[(w, h)]

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [9]:
model = ORTModelForCustomTasks.from_pretrained("./owlvit-large-patch14", file_name="onnx/model_quantized.onnx",cache_dir="./huggingface")
tokenizer = CLIPTokenizerFast.from_pretrained("Xenova/owlvit-large-patch14")
im_processor = OwlViTImageProcessor.from_pretrained("Xenova/owlvit-large-patch14")

The ONNX file onnx/model_quantized.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


/Users/tarik.setia/lab/cv-benchmark/.venv/lib/python3.10/site-packages/transformers/models/owlvit/feature_extraction_owlvit.py:28: FutureWarning: The class OwlViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use OwlViTImageProcessor instead.
  warnings.warn(


In [12]:
prompt = "human face"
text_inputs = tokenizer(prompt, padding="max_length", return_tensors='pt')
input_ids = text_inputs['input_ids']
attention_mask = text_inputs['attention_mask']
attention_mask, input_ids

(tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[49406,  2751,  1710, 49407,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0]]))

In [19]:
frames = im_processor([Image.fromarray(frame) for frame in video.values()], return_tensor="pt")["pixel_values"]


Unused or unrecognized kwargs: return_tensor.


In [20]:
frames[0].shape

(3, 840, 840)

In [21]:
results = {}

for idx, frame in enumerate(frames):
    detections = model(pixel_values=torch.Tensor([frame]), input_ids=input_ids, attention_mask=attention_mask)
    detections = im_processor.post_process_object_detection(detections, target_sizes=target_size)[0]
    results[idx] = detections
    print(detections)

/var/folders/3h/rh35kxps1_q0f42hby8yc0j8f320cp/T/ipykernel_38735/306929315.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  detections = model(pixel_values=torch.Tensor([frame]), input_ids=input_ids, attention_mask=attention_mask)


KeyboardInterrupt: 

In [7]:

import time
import torch
from utils.protocols import OwlVit
from utils.utils import convert_model_detection
from utils.video import read_video
from utils.utils import get_file_name, get_gpu_name
from loguru import logger
from datetime import datetime

MODEL= "owlvit-large-p14-hf"

text = "face"
BASE_DIR = "experiments/owlvit"

def run_model(frames):
    results = {}
    for frame_id, frame in frames.items():
        with torch.no_grad():
        #with torch.inference_mode():
            outputs = model(**frame)
            results[frame_id] = outputs
    return results

def process_video(video, frames=[]):
    start_time = datetime.now()
    target_sizes = None
    
    frames_ = {}
    start = time.time()
    for frame_id, frame in read_video(video):
        image = Image.fromarray(frame.astype("uint8"))
        inputs = processor(images=image, text=text, return_tensors="pt").to(device)
        frames_[frame_id] = inputs
        if target_sizes is None:
            target_sizes = torch.Tensor([image.size[::-1]])
    
    pre_processing_time = time.time() - start
    
    n_frames = len(frames_)
    
    start = time.time()
    outputs = run_model(frames_)
    inference_time = time.time() - start
    
    results = {}
    start = time.time()
    for frame_id, output in outputs.items():    
        detections = processor.post_process_object_detection(outputs=output, threshold=0.1, target_sizes=target_sizes)
        results[frame_id] = convert_model_detection(detections[0])
    post_processing_time = time.time() - start

    end_time = datetime.now()
    
    exp =  OwlVit(
        model=MODEL,
        gpu=get_gpu_name(),
        video_file=video,
        frames=None if not frames else frames,
        n_frames=n_frames,
        
        pre_processing_time=pre_processing_time,
        inference_time=inference_time,
        post_processing_time=post_processing_time,
        video_processing_time=(end_time-start_time).seconds,
       
        start_time=start_time.isoformat(),
        end_time=end_time.isoformat(),
        record_file=get_file_name(BASE_DIR, start, MODEL, video),
        
        data=results,
        prompt=text,
        
    )
    exp.save()
    exp.log()
    torch.cuda.empty_cache()
    return exp

In [8]:
results = []
for i in range(1):
    vid_hd = process_video("data/720.mp4")
    results.append(vid_hd)
    vid_fhd = process_video("data/1080.mp4")
    results.append(vid_fhd)

2024-06-12 00:39:12.774 | INFO     | __main__:process_video:26 - Reading Frame
2024-06-12 00:39:21.206 | INFO     | __main__:process_video:39 - nframes = 3
2024-06-12 00:39:21.207 | INFO     | __main__:process_video:40 - Running MOdel
2024-06-12 00:39:48.419 | INFO     | __main__:process_video:45 - Running Postprocessing
2024-06-12 00:39:48.421 | INFO     | __main__:process_video:53 - Done!
2024-06-12 00:39:48.423 | INFO     | utils.protocols:log:113 - data/720.mp4 | frames=3 | model_fps=0.11024676022476197 | inference_time=27.211683988571167 | preprocess_time=8.374552011489868
2024-06-12 00:39:48.429 | INFO     | __main__:process_video:26 - Reading Frame
2024-06-12 00:40:00.111 | INFO     | __main__:process_video:39 - nframes = 3
2024-06-12 00:40:00.112 | INFO     | __main__:process_video:40 - Running MOdel
2024-06-12 00:40:26.406 | INFO     | __main__:process_video:45 - Running Postprocessing
2024-06-12 00:40:26.408 | INFO     | __main__:process_video:53 - Done!
2024-06-12 00:40:26.4

In [9]:
columns = results[0].columns
rows = [result.row for result in results]

In [10]:
import pandas as pd

df = pd.DataFrame(rows, columns=columns)
df.head(10)

,model,gpu,video_file,batch_size,n_frames,pre_processing_fps,inference_fps,post_processing_fps,video_fps,pre_processing_time,inference_time,post_processing_time,video_processing_time,start_time,end_time,record_file,prompt,data
0,owlvit-large-p14-hf,cpu,data/720.mp4,1,3,0.358228,0.110247,1414.446043,0.085714,8.374552,27.211684,0.002121,35,2024-06-12T00:39:12.774421,2024-06-12T00:39:48.422247,experiments/owlvit/exp-owlvit-large-p14-hf-cpu...,face,"{0: [box=[366.6923522949219, 66.16351318359375..."
1,owlvit-large-p14-hf,cpu,data/1080.mp4,1,3,0.257969,0.114098,2181.881741,0.081081,11.629296,26.293289,0.001375,37,2024-06-12T00:39:48.429717,2024-06-12T00:40:26.408882,experiments/owlvit/exp-owlvit-large-p14-hf-cpu...,face,"{0: [box=[547.648193359375, 97.87193298339844,..."


In [11]:
from datetime import datetime
now = datetime.now()
csv_file = f"zz-{MODEL}-{get_gpu_name()}-{now.day}-{now.hour}-{now.minute}.csv"
df.to_csv(csv_file)